In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import math
import cv2
from keras.models import model_from_json, load_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.utils import *
from keras.metrics import AUC
from sklearn.utils import class_weight
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, classification_report
from stn import spatial_transformer_network as transformer

In [2]:
class STN(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(STN, self).__init__(**kwargs)
        self.pool1 = tf.keras.layers.MaxPool2D()
        self.conv1 = tf.keras.layers.Conv2D(32, [5, 5], activation='relu')
        self.pool2 = tf.keras.layers.MaxPool2D()
        self.conv2 = tf.keras.layers.Conv2D(64, [5, 5], activation='relu')
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(12, activation='relu')
        self.fc2 = tf.keras.layers.Dense(6, activation=None, bias_initializer=tf.keras.initializers.constant([1.0, 0.0, 0.0, 0.0, 1.0, 0.0]), kernel_initializer='zeros')

    def build(self, input_shape):
        print("Building Localization Network with input shape:", input_shape)

    def compute_output_shape(self, input_shape):
        return [None, 6]

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        theta = self.fc2(x)
        out = transformer(inputs, theta)
        return out

In [3]:
from keras.preprocessing import image

fer_list= ['anger', 'disgust', 'fear', 'happiness', 'sadness', 'surprise', 'neutral']
ck_list = ['anger', 'contempt', 'disgust', 'fear', 'happiness', 'neutral', 'sadness', 'surprise'] 
# race_list = ['white', 'black', 'asian', 'indian', 'others']
gender_list = ['male', 'female']
age_list = ['<20', '20-25', '25-30', '30-40', '40-60', '>60']

json_file_fer = open('saved_models/fer_model.json', 'r')
json_file_age = open('saved_models/age_utk_model.json', 'r')
json_file_race = open('saved_models/race_utk_model.json', 'r')
json_file_gender = open('saved_models/gender_utk_model.json', 'r')
json_file_ck = open('saved_models/CK+_model.json', 'r')

loaded_fer_json = json_file_fer.read()
json_file_fer.close()

loaded_age_json = json_file_age.read()
json_file_age.close()

loaded_race_json = json_file_race.read()
json_file_race.close()

loaded_gender_json = json_file_gender.read()
json_file_gender.close()

loaded_ck_json = json_file_ck.read()
json_file_ck.close()

fer_model = model_from_json(loaded_fer_json, custom_objects={'STN': STN})
age_model = model_from_json(loaded_age_json, custom_objects={'STN': STN})
# race_model = model_from_json(loaded_race_json, custom_objects={'STN': STN})
gender_model = model_from_json(loaded_gender_json, custom_objects={'STN': STN})
ck_model = model_from_json(loaded_ck_json, custom_objects={'STN': STN})

fer_model.load_weights("saved_models/best_weight_FER.h5")
age_model.load_weights("saved_models/best_weight_UTK_age.h5")
# race_model.load_weights("saved_models/best_weight_UTK_race.h5")
gender_model.load_weights("saved_models/best_weight_UTK_gender.h5")
ck_model.load_weights("saved_models/best_weight_CK+.h5")

print("Loaded model from disk")

face_detector = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        roi_gray_frame = gray_frame[y: y+w, x: x+h]
#         print(roi_gray_frame.shape)
        roi_gray_frame = cv2.resize(roi_gray_frame, (48, 48))
        image_pixels = img_to_array(roi_gray_frame)
        image_pixels = np.expand_dims(image_pixels, axis=0)
        image_pixels /= 255

        fer_prediction = fer_model.predict(image_pixels)
        age_prediction = age_model.predict(image_pixels)
#         race_prediction = race_model.predict(image_pixels)
        gender_prediction = gender_model.predict(image_pixels)
        ck_prediction = ck_model.predict(image_pixels)
        
        fer_maxindex = np.argmax(fer_prediction[0])
        age_maxindex = np.argmax(age_prediction[0])
#         race_maxindex = np.argmax(race_prediction[0])
        gender_maxindex = np.argmax(gender_prediction[0])
        ck_maxindex = np.argmax(ck_prediction[0])
        
        cv2.putText(frame, 'Emotions by FER: ' + fer_list[fer_maxindex], (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, 'Age: ' + age_list[age_maxindex], (int(x), int(y)-20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
#         cv2.putText(frame, 'Race: ' + race_list[race_maxindex], (int(x), int(y)-40), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, 'Gender: ' + gender_list[gender_maxindex], (int(x), int(y)-60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)
        cv2.putText(frame, 'Emotions by CK+: ' + ck_list[ck_maxindex], (int(x), int(y)-80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow('Age, Gender, and Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Building Localization Network with input shape: (None, 48, 48, 1)
Building Localization Network with input shape: (None, 48, 48, 1)
Building Localization Network with input shape: (None, 48, 48, 1)
Building Localization Network with input shape: (None, 48, 48, 1)
Loaded model from disk
1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 23ms/step


1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 22ms/step
